# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [158]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [159]:
#Your code here
df = pd.read_csv('homepage_actions.csv')
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [160]:
#Checking for total clicked
total_clicks = len(df)
total_clicks

8188

In [161]:
df["action"].value_counts()

view     6328
click    1860
Name: action, dtype: int64

In [162]:
views = sum(df['action'] == 'view')
clicks= sum(df['action'] == 'click')
viewers = action_counts.loc['view', 'num_unique'] - action_counts.loc['click', 'num_unique']
# using max() since you cant have negative clicks
clickers = max(0, action_counts.loc['click', 'num_unique'] - action_counts.loc['view', 'num_unique'])

print("Number of Viewers:", viewers)
print("Number of Clickers with no views :",clickers)
print(f"The views {views} and clicks {clicks} add up to {views + clicks}, the total amount clicks.")
print("So all who clicked viewed the page.")

Number of Viewers: 4468
Number of Clickers with no views : 0
The views 6328 and clicks 1860 add up to 8188, the total amount clicks.
So all who clicked viewed the page.


In [163]:
experimentids = set(df[df.group=='experiment']['id'].unique())
controlids = set(df[df.group=='control']['id'].unique())
overlap_count = len(experimentids & controlids)
overlap_count

0

In [164]:
"""
No overlaps between the control group and experiemental group.
"""

'\nNo overlaps between the control group and experiemental group.\n'

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [165]:
from scipy import stats
from scipy.stats import t
import numpy as np

In [166]:
#Your code here
# Control Group Information
control_group = df[df['group'] == 'control']['id'].nunique()
control_clicks = df[(df['group'] == 'control') & (df['action'] == 'click')].shape[0]
control_mean = control_clicks/control_group

print(f"Control Group Amount:", control_group)
print(f"Control Group Clicks", control_clicks)
print(f"Control Mean:", control_mean)

Control Group Amount: 3332
Control Group Clicks 932
Control Mean: 0.2797118847539016


In [167]:
# Experimental Group Information
experiment_group = df[df['group'] == 'experiment']['id'].nunique()
experiment_clicks = df[(df['group'] == 'experiment') & (df['action'] == 'click')].shape[0]
experimental_mean = experiment_clicks/experiment_group
print(f"Experimental Group Amount:", experiment_group)
print(f"Experimental Group Clicks:", experiment_clicks)
print(f"Experimental Mean:", experimental_mean) 

Experimental Group Amount: 2996
Experimental Group Clicks: 928
Experimental Mean: 0.3097463284379172


In [168]:
# finding t
numer = control_mean - experimental_mean
denom = np.sqrt((control_clicks / control_group)**2 / control_group + (experiment_clicks / experiment_group)**2 / experiment_group)
t_value = numer / denom
t_value

-4.031391180357199

In [193]:
# finding df
s1_squared = (control_clicks / control_group) * (1 - (control_clicks / control_group))
s2_squared = (experiment_clicks / experiment_group) * (1 - (experiment_clicks / experiment_group))

n1 = control_group
n2 = experiment_group

numerator = (s1_squared / n1 + s2_squared / n2)**2
denominator = (s1_squared**2 / (n1**2 * (n1 - 1))) + (s2_squared**2 / (n2**2 * (n2 - 1)))
dof = numerator / denominator
dof


6211.177058990254

In [196]:
#Find P value with t value and dof
p_value = 1-stats.t.cdf(np.abs(t_value), dof)
p_value

2.805551363593839e-05

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [180]:
#Your code here

expected_experiment_clicks = control_mean * experiment_group
print(expected_experiment_clicks)

838.0168067226891


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [182]:
#Your code here
p = control_mean
n = experiment_group
var = p * n  * (1-p)
std = np.sqrt(var)
zscore = (experiment_clicks - expected_experiment_clicks)/std
zscore

3.6625360854823588

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [187]:
#Your code here
p_val =stats.norm.sf(zscore)
print(p_val)

0.00012486528006951198


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

In [197]:
"""
Although they are different, my found p value, and the calculated pvalue based off z_score, both
show that we can reject the null hypothesis. The experimental homepage is more effective.
"""

'\nAlthough they are different, my found p value, and the calculated pvalue based off z_score, both\nshow that we can reject the null hypothesis. \nThe experimental homepage is more effective.\n'

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.